## Data preparation

We have a corpus of messages (appear to be translated from their original language in some cases) and a separate file tagging categories for each message. 

Both files have an `id` column, which can be used to merge them. This column needs to be deduplicated first.

`genre` has the following values:
* direct (this is the only category with translated messages)
* news (most common category)
* social (least common)

I'll hang on to it as a potential feature.


In [1]:
# Libraries
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table

In [67]:
# Load data
messages = pd.read_csv("data/messages.csv")
categories = pd.read_csv("data/categories.csv")

# Remove duplicates
m_dupes = messages.id.duplicated()
messages.drop(messages.index[m_dupes], inplace=True)
messages.set_index('id', inplace=True)
print(messages.shape)

c_dupes = categories.id.duplicated()
categories.drop(categories.index[c_dupes], inplace=True)
categories.set_index('id', inplace=True)
print(categories.shape)

(26180, 3)
(26180, 1)


There are two fields for the message:
* `message` contains the message in English, and is what I'll be using for NLP
* `original` is the message in the original language; typically when the message is in English this will be NaN, but occasionally will have the same text as `message`

I'll use these to create a feature for whether the message has been translated.

In [69]:
# Make feature for translated messages
eng_msg = (messages.message == messages.original) | \
    (messages.original.isna())
eng_msg = [0 if val else 1 for val in eng_msg]
messages['translated'] = eng_msg

# Merge
df = messages.merge(categories, left_index = True, right_index = True)
df.head()

,message,original,genre,translated,categories
id,,,,,
2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,related-1;request-0;offer-0;aid_related-0;medi...
7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,related-1;request-0;offer-0;aid_related-1;medi...
8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,related-1;request-0;offer-0;aid_related-0;medi...
9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,related-1;request-1;offer-0;aid_related-1;medi...
12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,related-1;request-0;offer-0;aid_related-0;medi...


The tagging categories are present as a single string in the `categories` column and need to be unpacked into separate columns.  They are separated by a semicolon (';'), with the name of the category, a dash ('-') and the value. For all categories except for "related", the value is either 1 or 0. For some reason, the value of "2" is possible in "related".

The corpus contains no examples of certain categories. I'm dropping those for cleanliness. It might be worth revisiting very low incidence categories, particularly if they prove hard to model.

In [47]:
# Split message categories (convert into integers)
cat_cols = [col[:-2] for col in df.loc[2, 'categories'].split(';')]
cat_str = df.categories.replace(r'[^012;]', '', regex=True)
cat_vals = cat_str.str.split(';', expand=True).astype('int')
cat_vals.columns = cat_cols

# Remove labels with no instances
cat_vals = cat_vals.loc[:, cat_vals.sum() > 0]

In [13]:
# Join message data to the dataframe
df = pd.concat([df, cat_vals], axis = 1, sort = False)
df.drop(columns=['original', 'categories'], inplace=True)
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
id,,,,,,,,,,,,,,,,,,,,,
2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
8,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Data are exported to a SQLite database in the `data/` folder.

In [15]:
engine = create_engine('sqlite:///data/DisasterResponse.db')

tn = 'scored_messages'

if engine.dialect.has_table(engine, tn):
    meta = MetaData()
    tbl = Table(tn, meta)
    tbl.drop(engine)

df.to_sql(tn, engine, index=False)